# Programmatic usage of NeoFox

NeoFox provides an Application Programming Interface (API) that enables the integration into other applications. This API relies heavily on Protocol Buffers data models that provide placeholder objects to store the required data while enabling different representations, data manipulation, validation and normalization. We use the Protocol Buffers data models to generate Python code automatically and to implement validation and normalization around them, but Protocol Buffers is technology agnostic thus this may facilitate the integration with third party applications not necessarily implemented in Python (see https://developers.google.com/protocol-buffers). The API is tightly integrated with the Python data analysis library Pandas (see https://pandas.pydata.org/).

Here we show: 

* how to create new model objects
* how to import/export these objects into different representations
* how to manipulate them
* how to validate and normalize on the data 

And finally we show how to run NeoFox programmatically, you may want to skip to this part for a quick grasp of the API usage.

## Neoantigens

The neoantigen is the central piece of information that NeoFox handles, all output annotations refer to a neoantigen. A neoantigen is formed by two subentities transcript and mutation, plus some additional attributes. Here we show how to create a neoantigen, transform it into different representations and validate it.

### Create a neoantigen


In [1]:
from neofox.model.neoantigen import Neoantigen, Mutation

Create a mutation:

In [3]:
mutation = Mutation(
    wild_type_xmer="DEVLGEPSQDILVIDQTRLEATISPET", 
    mutated_xmer="DEVLGEPSQDILVTDQTRLEATISPET")

Create a neoantigen using the previous transcript and mutation:

In [4]:
neoantigen = Neoantigen(
    patient_identifier="P123", 
    mutation=mutation, 
    gene="VCAN", 
    rna_expression=0.519506894, 
    rna_variant_allele_frequency=0.857142857, 
    dna_variant_allele_frequency=0.294573643)

### Representation into different formats

The same piece of data agreeing with NeoFox data models can be represented in different formats. Here we show how to transform the data between several formats: JSON, Python dictionaries, Protocol Buffers binary representations, Pandas dataframes and tabular representations in files. This is relevant for enabling data import and export and adding flexibility to the integration with other tools.

What is shown here is applicable to all entities in NeoFox data models.

These objects can be easily transformed into JSON:

In [5]:
print(neoantigen.to_json(indent=2))

{
  "patientIdentifier": "P123",
  "gene": "VCAN",
  "mutation": {
    "wildTypeXmer": "DEVLGEPSQDILVIDQTRLEATISPET",
    "mutatedXmer": "DEVLGEPSQDILVTDQTRLEATISPET"
  },
  "rnaExpression": 0.519506894,
  "dnaVariantAlleleFrequency": 0.294573643,
  "rnaVariantAlleleFrequency": 0.857142857
}


They can also be transformed into a Python native dictionary:

In [6]:
neoantigen.to_dict()

{'patientIdentifier': 'P123',
 'gene': 'VCAN',
 'mutation': {'wildTypeXmer': 'DEVLGEPSQDILVIDQTRLEATISPET',
  'mutatedXmer': 'DEVLGEPSQDILVTDQTRLEATISPET'},
 'rnaExpression': 0.519506894,
 'dnaVariantAlleleFrequency': 0.294573643,
 'rnaVariantAlleleFrequency': 0.857142857}

And also into the Protocol Buffers binary format that allows a better compression for storing the data or sending it over the wire:

In [7]:
neoantigen.SerializeToString()

b'\x12\x04P123\x1a\x04VCAN":\x12\x1bDEVLGEPSQDILVIDQTRLEATISPET\x1a\x1bDEVLGEPSQDILVTDQTRLEATISPET-g\xfe\x04?=[\xd2\x96>E\xb7m[?'

### Integration with Pandas

NeoFox integrates with the Python library for data analysis Pandas (see https://pandas.pydata.org/). A single object can be transformed into a Pandas `Series` and a list of objects can be transformed into a Pandas `DataFrame`. Pandas provide functionality to persist this tabular representations to files that can be stored and imported into other environments, for instance R.

What is shown here is applicable to all entities in NeoFox data models.

In [8]:
import pandas as pd
from neofox.model.conversion import ModelConverter

Transform a neoantigen into a Pandas `Series`:

In [9]:
neoantigen_series = ModelConverter.object2series(neoantigen)
neoantigen_series

identifier                                                 
patient_identifier                                     P123
gene                                                   VCAN
rna_expression                                     0.519507
imputed_gene_expression                                   0
dna_variant_allele_frequency                       0.294574
rna_variant_allele_frequency                       0.857143
mutation.position                                        []
mutation.wild_type_xmer         DEVLGEPSQDILVIDQTRLEATISPET
mutation.mutated_xmer           DEVLGEPSQDILVTDQTRLEATISPET
Name: 0, dtype: object

Transform a list of transcripts into a Pandas `DataFrame`:

In [10]:
mutation2 = Mutation(
    wild_type_xmer="AAAAAAAAAAAAAAAAAAAAAAAAAAA", 
    mutated_xmer="AAAAAAAAAAAAAGAAAAAAAAAAAAA")
mutations_df = ModelConverter.objects2dataframe([mutation, mutation2])
mutations_df

,position,wildTypeXmer,mutatedXmer
0,[],DEVLGEPSQDILVIDQTRLEATISPET,DEVLGEPSQDILVTDQTRLEATISPET
1,[],AAAAAAAAAAAAAAAAAAAAAAAAAAA,AAAAAAAAAAAAAGAAAAAAAAAAAAA


Persist any Pandas object into a file:

In [11]:
mutations_df.to_csv("my_mutations.csv", sep="\t", index=False)

And read it back:

In [12]:
mutations_df2 = pd.read_csv("my_mutations.csv", sep="\t")
mutations = []
for _, row in mutations_df2.iterrows():
    #nested_dict = ModelConverter._flat_dict2nested_dict(flat_dict=row.to_dict())
    mutations.append(Mutation().from_dict(row.to_dict()))
#ModelConverter.objects2dataframe(mutations)
mutations

[Mutation(position='[]', wild_type_xmer='DEVLGEPSQDILVIDQTRLEATISPET', mutated_xmer='DEVLGEPSQDILVTDQTRLEATISPET'),
 Mutation(position='[]', wild_type_xmer='AAAAAAAAAAAAAAAAAAAAAAAAAAA', mutated_xmer='AAAAAAAAAAAAAGAAAAAAAAAAAAA')]

In some cases you will may be handling nested objects, for instance a neoantigen. The nesting is flattened into the DataFrame by concatenating field names with a dot, eg: `mutation.wild_type_xmer`. In order to read the flattened data back into the nested models we need to add an intermediate step.

In [13]:
# the flattened dictionary
neoantigen_series.to_dict()

{'identifier': '',
 'patient_identifier': 'P123',
 'gene': 'VCAN',
 'rna_expression': 0.519506894,
 'imputed_gene_expression': 0.0,
 'dna_variant_allele_frequency': 0.294573643,
 'rna_variant_allele_frequency': 0.857142857,
 'mutation.position': [],
 'mutation.wild_type_xmer': 'DEVLGEPSQDILVIDQTRLEATISPET',
 'mutation.mutated_xmer': 'DEVLGEPSQDILVTDQTRLEATISPET'}

In [14]:
# the nested dictionary
ModelConverter._flat_dict2nested_dict(flat_dict=neoantigen_series.to_dict())

{'identifier': '',
 'patient_identifier': 'P123',
 'gene': 'VCAN',
 'rna_expression': 0.519506894,
 'imputed_gene_expression': 0.0,
 'dna_variant_allele_frequency': 0.294573643,
 'rna_variant_allele_frequency': 0.857142857,
 'mutation': {'position': [],
  'wild_type_xmer': 'DEVLGEPSQDILVIDQTRLEATISPET',
  'mutated_xmer': 'DEVLGEPSQDILVTDQTRLEATISPET'}}

In [15]:
# we can load the nested dictionary into a nested model object
Neoantigen().from_dict(ModelConverter._flat_dict2nested_dict(flat_dict=neoantigen_series.to_dict()))

Neoantigen(identifier='', patient_identifier='P123', gene='VCAN', mutation=Mutation(position=[], wild_type_xmer='DEVLGEPSQDILVIDQTRLEATISPET', mutated_xmer='DEVLGEPSQDILVTDQTRLEATISPET'), rna_expression=0.519506894, imputed_gene_expression=0.0, dna_variant_allele_frequency=0.294573643, rna_variant_allele_frequency=0.857142857)

### Data validation

The quality and cleanliness of data is of great importance to enable an effective data analysis and make the data machine readable. Clean data means that the data is valid and that it is in a normal and homogeneous form. The use of controlled vocabularies help to represent knowledge in a standardised way. This is a domain specific task, although it can be assisted with the right tools such as Pandas in Python or tidyverse in R, it requires domain expertise to perform it. NeoFox provides this domain expertise out of the box with its validation and normalization layers on top of its data models.

In [16]:
from neofox.model.conversion import ModelValidator
from neofox.exceptions import NeofoxDataValidationException

The data validation checks for missing required fields and shows relevant messages.

In [17]:
try:
    ModelValidator.validate_neoantigen(neoantigen=Neoantigen())
except NeofoxDataValidationException as e:
    print("Error message: {}".format(e))

[E 210602 14:43:36 conversion:549] {}


Error message: A patient identifier is missing. Please provide patientIdentifier in the input file


It also performs more domain specific validations such as aminoacids being valid according to the IUPAC standard aminoacid representation.

In [18]:
try:
    ModelValidator.validate_neoantigen(neoantigen=Neoantigen(
        patient_identifier="12345",
        gene="VCAN",
        mutation=Mutation(
            wild_type_xmer="123456AAAAAAAAAAAAAA", # wrong aminoacid representation
            mutated_xmer="123456GAAAAAAAAAAAAA")))
except NeofoxDataValidationException as e:
    print("Error message: {}".format(e))

[E 210602 14:43:44 conversion:549] {
       "patientIdentifier": "12345",
       "gene": "VCAN",
       "mutation": {
          "wildTypeXmer": "123456AAAAAAAAAAAAAA",
          "mutatedXmer": "123456GAAAAAAAAAAAAA"
       }
    }


Error message: Non existing aminoacid 1


The data normalization layer ensures the aminoacid representation is normalized into 1 letter IUPAC codes.

In [19]:
valid_neoantigen = ModelValidator.validate_neoantigen(neoantigen=Neoantigen(
    patient_identifier="12345",
    mutation=Mutation(
        wild_type_xmer="AAAAAAAAAAAAA",
        mutated_xmer="aaaaaGaaaaa")))

print(valid_neoantigen.mutation.to_json(indent=2))

{
  "position": [
    6
  ],
  "wildTypeXmer": "AAAAAAAAAAAAA",
  "mutatedXmer": "AAAAAGAAAAA"
}


After validation a unique neoantigen identifier is generated, this is a hash function of the normalized neoantigen representation, thus two different representations of the same neoantigen will share the same identifier after normalization.

In [20]:
validated_neoantigen = ModelValidator.validate_neoantigen(neoantigen=neoantigen)
print(validated_neoantigen.to_json(indent=2))

{
  "identifier": "v9cO6QEVw9XjrpJlvJ3isw==",
  "patientIdentifier": "P123",
  "gene": "VCAN",
  "mutation": {
    "position": [
      14
    ],
    "wildTypeXmer": "DEVLGEPSQDILVIDQTRLEATISPET",
    "mutatedXmer": "DEVLGEPSQDILVTDQTRLEATISPET"
  },
  "rnaExpression": 0.519506894,
  "dnaVariantAlleleFrequency": 0.294573643,
  "rnaVariantAlleleFrequency": 0.857142857
}


## Patients

The neoantigen annotation process needs some context information, in particular some data about the individual where the somatic mutation creating this neoantigen took place. This information includes mainly the HLA types of the patient which is needed to compute the binding of the potential neoepitopes.

### Parse MHC I alleles into a normal representation

The main complexity in the patient model is the representation of the MHC I and MHC II alleles present in the patient. The HLA alleles are typically represented using the nomenclature defined here http://hla.alleles.org, but de facto there is certain flexibility in the representation of HLA alleles in the community. NeoFox aims at normalizing the different HLA representations into a controlled representation agreeing with the HLA nomenclature. NeoFox only supports the classic MHC genes and although the provided HLA type is kept internally it only works with the first 4 digits.

There are specific functions in NeoFox to parse a list of non normal HLA alleles into a normalized representation of the HLA alleles. Due to the heterogeneous representations of alleles we use IPD-IMGT/HLA database in order to normalize ambiguous alleles (e.g.: B15228=>HLA-B*15:228 and DPB110401=>HLA-DPB1*104:01).

Furthermore, the zygosity of each HLA gene is inferred.

In [33]:
from neofox.references.references import ReferenceFolder
import os

os.environ["NEOFOX_REFERENCE_FOLDER"] = "/neofox_install/reference_data"
reference_folder = ReferenceFolder()
hla_database = reference_folder.get_hla_database()

[I 210602 14:57:45 references:189] Reference genome folder: /neofox_install/reference_data
[I 210602 14:57:45 references:190] Resources
[I 210602 14:57:45 references:192] /neofox_install/reference_data/netmhc2pan_available_alleles.txt
[I 210602 14:57:45 references:192] /neofox_install/reference_data/netmhcpan_available_alleles.txt
[I 210602 14:57:45 references:192] /neofox_install/reference_data/iedb
[I 210602 14:57:45 references:192] /neofox_install/reference_data/proteome_db
[I 210602 14:57:45 references:192] /neofox_install/reference_data/proteome_db/Homo_sapiens.fa
[I 210602 14:57:45 references:192] /neofox_install/reference_data/iedb/IEDB.fasta
[I 210602 14:57:45 references:192] /neofox_install/reference_data/proteome_db/Homo_sapiens.fa
[I 210602 14:57:45 references:192] /neofox_install/reference_data/hla_database_allele_list.csv


Parse a list of MHC I alleles. The data validation will ensure that the data is valid and it will infer the zygosity of the different genes. The data normalization layer will normalize the HLA representation into a the valid HLA nomenclature including the first 4 digits. Different representations of the same allele will be matched after normalization.

In [55]:
mhc1 = ModelConverter.parse_mhc1_alleles(
    ["HLA-A*01:01:02:03N", "HLA-A*01:02:02:03N", "B15228", "HLA-B*15:228:02:04N", "C03_163"], 
    hla_database=hla_database)
ModelConverter.objects2dataframe(mhc1)

,name,zygosity,alleles
0,A,HETEROZYGOUS,"[{'fullName': 'HLA-A*01:01:02:03N', 'name': 'H..."
1,B,HOMOZYGOUS,"[{'fullName': 'HLA-B*15:228', 'name': 'HLA-B*1..."
2,C,HEMIZYGOUS,"[{'fullName': 'HLA-C*03:163', 'name': 'HLA-C*0..."


In [56]:
ModelConverter.objects2dataframe(mhc1[0].alleles + mhc1[1].alleles + mhc1[2].alleles)

,fullName,name,gene,group,protein
0,HLA-A*01:01:02:03N,HLA-A*01:01,A,01,01
1,HLA-A*01:02:02:03N,HLA-A*01:02,A,01,02
2,HLA-B*15:228,HLA-B*15:228,B,15,228
3,HLA-C*03:163,HLA-C*03:163,C,03,163


### Validation and normalization of MHC alleles

The data validation layer checks that the provided allele representations are valid.

In [41]:
try:
    ModelConverter.parse_mhc1_alleles(["HLA-W*01:01:02:03N"], hla_database=hla_database)  # bad gene W
except NeofoxDataValidationException as e:
    print ("Error message: {}".format(e))

Error message: Allele does not match HLA allele pattern HLA-W*01:01:02:03N


In [37]:
try:
    ModelConverter.parse_mhc1_alleles(["HLA-A*first:second:02:03N"], hla_database=hla_database)  # bad allele representation
except NeofoxDataValidationException as e:
    print ("Error message: {}".format(e))

Error message: Allele does not match HLA allele pattern HLA-A*first:second:02:03N


In [57]:
try:
    ModelConverter.parse_mhc1_alleles(["HLA-A*01:02:02:03N", "HLA-A*01:03:02:03N", "HLA-A*01:04:02:03N"], hla_database=hla_database)  # wrong number of alleles
except NeofoxDataValidationException as e:
    print ("Error message: {}".format(e))

Error message: More than 2 alleles for gene A


A warning message will be shown for non existing HLA alleles.

In [58]:
ModelConverter.parse_mhc1_alleles(["HLA-B*01:02:02:03N", "HLA-C*01:02"], hla_database=hla_database)

[W 210602 15:09:17 mhc_parser:68] Allele HLA-B*01:02:02:03N does not exist in the HLA database


[Mhc1(name=<Mhc1Name.A: 0>, zygosity=<Zygosity.LOSS: 3>, alleles=[]),
 Mhc1(name=<Mhc1Name.B: 1>, zygosity=<Zygosity.HEMIZYGOUS: 2>, alleles=[MhcAllele(full_name='HLA-B*01:02:02:03N', name='HLA-B*01:02', gene='B', group='01', protein='02')]),
 Mhc1(name=<Mhc1Name.C: 2>, zygosity=<Zygosity.HEMIZYGOUS: 2>, alleles=[MhcAllele(full_name='HLA-C*01:02', name='HLA-C*01:02', gene='C', group='01', protein='02')])]

### Parse MHC II alleles into a normal representation

The model for MHC II alleles is more complex as we need to reflect all combinations of alpha and beta chains, but the data validation and normalization provided by NeoFox is fundamentally the same.

Parse a list of MHC II alleles:

In [60]:
mhc2 = ModelConverter.parse_mhc2_alleles(["HLA-DPA1*01:03", "HLA-DPA1*01:04", "HLA-DPB1*01:01", "HLA-DPB1*01:01", 
                                          "HLA-DQA1*01:01", "HLA-DQA1*01:01", "HLA-DQB1*02:01", "HLA-DQB1*02:01", 
                                          "HLA-DRB1*01:01", "HLA-DRB1*01:01"], hla_database=hla_database)

An MHC II gene with an heteroyzgous alpha chain and an homozygous beta chain has two isoforms

In [61]:
mhc2[1].to_dict()

{'name': 'DP',
 'genes': [{'name': 'DPA1',
   'zygosity': 'HETEROZYGOUS',
   'alleles': [{'fullName': 'HLA-DPA1*01:03',
     'name': 'HLA-DPA1*01:03',
     'gene': 'DPA1',
     'group': '01',
     'protein': '03'},
    {'fullName': 'HLA-DPA1*01:04',
     'name': 'HLA-DPA1*01:04',
     'gene': 'DPA1',
     'group': '01',
     'protein': '04'}]},
  {'name': 'DPB1',
   'alleles': [{'fullName': 'HLA-DPB1*01:01',
     'name': 'HLA-DPB1*01:01',
     'gene': 'DPB1',
     'group': '01',
     'protein': '01'}]}],
 'isoforms': [{'name': 'HLA-DPA1*01:03-DPB1*01:01',
   'alphaChain': {'fullName': 'HLA-DPA1*01:03',
    'name': 'HLA-DPA1*01:03',
    'gene': 'DPA1',
    'group': '01',
    'protein': '03'},
   'betaChain': {'fullName': 'HLA-DPB1*01:01',
    'name': 'HLA-DPB1*01:01',
    'gene': 'DPB1',
    'group': '01',
    'protein': '01'}},
  {'name': 'HLA-DPA1*01:04-DPB1*01:01',
   'alphaChain': {'fullName': 'HLA-DPA1*01:04',
    'name': 'HLA-DPA1*01:04',
    'gene': 'DPA1',
    'group': '01',
   

An MHC II gene with an homozygous alpha and beta chains has a single isoform.

In [62]:
mhc2[2].to_dict()

{'name': 'DQ',
 'genes': [{'name': 'DQA1',
   'alleles': [{'fullName': 'HLA-DQA1*01:01',
     'name': 'HLA-DQA1*01:01',
     'gene': 'DQA1',
     'group': '01',
     'protein': '01'}]},
  {'name': 'DQB1',
   'alleles': [{'fullName': 'HLA-DQB1*02:01',
     'name': 'HLA-DQB1*02:01',
     'gene': 'DQB1',
     'group': '02',
     'protein': '01'}]}],
 'isoforms': [{'name': 'HLA-DQA1*01:01-DQB1*02:01',
   'alphaChain': {'fullName': 'HLA-DQA1*01:01',
    'name': 'HLA-DQA1*01:01',
    'gene': 'DQA1',
    'group': '01',
    'protein': '01'},
   'betaChain': {'fullName': 'HLA-DQB1*02:01',
    'name': 'HLA-DQB1*02:01',
    'gene': 'DQB1',
    'group': '02',
    'protein': '01'}}]}

The MHC II DRB gene is a special case with no alpha chain represented as this is not variable.

In [63]:
mhc2[0].to_dict()

{'genes': [{'alleles': [{'fullName': 'HLA-DRB1*01:01',
     'name': 'HLA-DRB1*01:01',
     'gene': 'DRB1',
     'group': '01',
     'protein': '01'}]}],
 'isoforms': [{'name': 'HLA-DRB1*01:01',
   'betaChain': {'fullName': 'HLA-DRB1*01:01',
    'name': 'HLA-DRB1*01:01',
    'gene': 'DRB1',
    'group': '01',
    'protein': '01'}}]}

Beware that incomplete MHC II molecules missing one of the chains will have no isoforms and thus no binding will be computed on them. In the case below the beta chain allele for the DP gene is missing.

In [66]:
mhc2 = ModelConverter.parse_mhc2_alleles(["HLA-DPA1*01:03", "HLA-DPA1*01:04"], hla_database=hla_database)
mhc2[1].to_dict()

{'name': 'DP',
 'genes': [{'name': 'DPA1',
   'zygosity': 'HETEROZYGOUS',
   'alleles': [{'fullName': 'HLA-DPA1*01:03',
     'name': 'HLA-DPA1*01:03',
     'gene': 'DPA1',
     'group': '01',
     'protein': '03'},
    {'fullName': 'HLA-DPA1*01:04',
     'name': 'HLA-DPA1*01:04',
     'gene': 'DPA1',
     'group': '01',
     'protein': '04'}]},
  {'name': 'DPB1', 'zygosity': 'LOSS'}]}

### Create a patient

In [72]:
from neofox.model.neoantigen import Patient


mhc1 = ModelConverter.parse_mhc1_alleles(["HLA-A*01:01:02:03N", "HLA-A*01:02:02:03N", 
                                          "HLA-B*15:01:02:03N", "HLA-B*15:01:02:04N", 
                                          "HLA-C*03:02"], hla_database=hla_database)
mhc2 = ModelConverter.parse_mhc2_alleles(["HLA-DPA1*01:03", "HLA-DPA1*01:04", "HLA-DPB1*01:01", "HLA-DPB1*01:01", 
                                          "HLA-DQA1*01:01", "HLA-DQA1*01:01", "HLA-DQB1*02:01", "HLA-DQB1*02:01", 
                                          "HLA-DRB1*01:01", "HLA-DRB1*01:01"], hla_database=hla_database)
patient = Patient(
    identifier="P123", 
    is_rna_available=True, 
    tumor_type="NSCLC", 
    mhc1=mhc1,
    mhc2=mhc2
)
ModelConverter.object2series(patient)

identifier                                                       P123
is_rna_available                                                 True
tumor_type                                                      NSCLC
mhc1                [{'name': 'A', 'zygosity': 'HETEROZYGOUS', 'al...
mhc2                [{'name': 'DR', 'genes': [{'name': 'DRB1', 'zy...
Name: 0, dtype: object

### Validate a patient

In [73]:
validated_patient = ModelValidator.validate_patient(patient)

A patient requires an identifier. MHC I and MHC II are optional in case one or the other are not available, the output annotations are adapted accordingly.

In [74]:
try:
    ModelValidator.validate_patient(Patient())  # missing patient identifier
except NeofoxDataValidationException as e:
    print ("Error message: {}".format(e))

[E 210602 15:14:20 conversion:590] {}


Error message: A patient identifier is missing


In [75]:
patient_without_mhc2 = ModelValidator.validate_patient(Patient(identifier="12345", mhc1=mhc1))

In [76]:
patient_without_mhc1 = ModelValidator.validate_patient(Patient(identifier="12345", mhc2=mhc2))

## Run Neofox

### Parse input data from a file

Although we could create the data objects manually as shown above, for convenience it is useful to store the data in tabular format. Here we show how to parse the neoantigens and patients from tabular files. 

The tabular file for neoantigens should look as follows:

In [77]:
pd.read_csv("test_model_file.txt", sep="\t")

,gene,transcript_identifier,mutation.mutatedXmer,mutation.wildTypeXmer,patientIdentifier
0,VCAN,uc003kii.3,DEVLGEPSQDILVTDQTRLEATISPET,DEVLGEPSQDILVIDQTRLEATISPET,Pt27
1,DCST2,uc001fgm.3,RTNLLAALHRSVRWRAADQGHRSAFLV,RTNLLAALHRSVRRRAADQGHRSAFLV,Pt24
2,NRAS,uc009wgu.3,MTEYKLVVVGACGVGKSALTIQLIQ,MTEYKLVVVGAGGVGKSALTIQLIQ,Pt28
3,CEP350,uc001gnt.3,QTDSSSSDMQACSKDKAKISLGSSIDS,QTDSSSSDMQACSQDKAKISLGSSIDS,Pt63
4,CPPED1,uc002dca.4,DRAIPLVLVSGNHYIGNTPTAETVEEF,DRAIPLVLVSGNHDIGNTPTAETVEEF,Pt77
5,CXorf26,uc004ecl.1,YNKAVYISVQDKEEEKGVNNGGEKRAD,YNKAVYISVQDKEGEKGVNNGGEKRAD,Pt117
6,IGSF9B,uc001qgx.4,ASTHLTVIGTSPHVPGSVRVQVSMTTA,ASTHLTVIGTSPHAPGSVRVQVSMTTA,Pt110
7,HEATR5A,uc001wrf.4,TRRDEKSHPFTNPQWATRVFAAECVCR,TRRDEKSHPFTNPRWATRVFAAECVCR,Pt26
8,CHRDL2,uc001ovh.3,ARPDMFCLFHGKRHFPGESWHPYLEPQ,ARPDMFCLFHGKRYFPGESWHPYLEPQ,Pt77


There is a specific function to parse an input file into a list of neoantigens. Any additional column not matching a field in the neoantigens model, in this case `transcript_identifier`, will be parsed into the external annotations. Neofox when executed from the command line interface adds these external annotations in the output together with the new annotations.

In [79]:
neoantigens, external_annotations = ModelConverter.parse_neoantigens_file("test_model_file.txt", sep="\t")

The tabular file for patients should look as follows:

In [80]:
pd.read_csv("test_patient_file.txt", sep="\t")

,identifier,mhcIAlleles,mhcIIAlleles,isRnaAvailable,tumorType
0,Pt27,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
1,Pt24,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
2,Pt28,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
3,Pt63,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
4,Pt77,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
5,Pt117,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
6,Pt110,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC
7,Pt26,"HLA-A*03:01,HLA-A*29:02,HLA-B*07:02,HLA-B*44:0...","HLA-DRB1*04:02,HLA-DRB1*08:01,HLA-DQA1*03:01,H...",True,HNSC


Parse the patients into the model objects as follows:

In [84]:
patients = ModelConverter.parse_patients_file("test_patient_file.txt", hla_database=hla_database)

### Annotate your neoantigens

Running NeoFox requires the configuration its configuration through a number of environment variables, this is described in detail elsewhere in the documentation. This configuration can also be provided through a file passed into Neofox class in the field `configuration_file`.

In [85]:
from neofox.neofox import NeoFox
import os

In [86]:
os.environ["NEOFOX_REFERENCE_FOLDER"] = "/neofox_install/reference_data/"
os.environ["NEOFOX_RSCRIPT"] = "/usr/bin/Rscript"
os.environ["NEOFOX_BLASTP"] = "/neofox_install/ncbi-blast-2.10.1+/bin/blastp"
os.environ["NEOFOX_NETMHCPAN"] = "/neofox_install/netMHCpan-4.1/netMHCpan"
os.environ["NEOFOX_NETMHC2PAN"] = "/neofox_install/netMHCIIpan-4.0/netMHCIIpan"
os.environ["NEOFOX_MIXMHCPRED"] = "/neofox_install/MixMHCpred-2.1/MixMHCpred"
os.environ["NEOFOX_MIXMHC2PRED"] = "/neofox_install/MixMHC2pred-1.2/MixMHC2pred_unix"
os.environ["NEOFOX_PRIME"] = "/neofox_install/PRIME-master/PRIME"
annotations = NeoFox(neoantigens=neoantigens, patients=patients, num_cpus=4).get_annotations()

[I 210602 15:27:32 references:189] Reference genome folder: /neofox_install/reference_data/
[I 210602 15:27:32 references:190] Resources
[I 210602 15:27:32 references:192] /neofox_install/reference_data/netmhc2pan_available_alleles.txt
[I 210602 15:27:32 references:192] /neofox_install/reference_data/netmhcpan_available_alleles.txt
[I 210602 15:27:32 references:192] /neofox_install/reference_data/iedb
[I 210602 15:27:32 references:192] /neofox_install/reference_data/proteome_db
[I 210602 15:27:32 references:192] /neofox_install/reference_data/proteome_db/Homo_sapiens.fa
[I 210602 15:27:32 references:192] /neofox_install/reference_data/iedb/IEDB.fasta
[I 210602 15:27:32 references:192] /neofox_install/reference_data/proteome_db/Homo_sapiens.fa
[I 210602 15:27:32 references:192] /neofox_install/reference_data/hla_database_allele_list.csv
[I 210602 15:27:33 expression_imputation:70] Fetching the gene expression at VCAN:10
[I 210602 15:27:33 expression_imputation:76] Fetched a gene express

AttributeError: 'NoneType' object has no attribute 'affinity_score'

Neofox returns a list of annotations for each neoantigen, these are stored in an object called `NeoantigenAnnotations` which contains the corresponding neoantigen identifier, the annotator (ie: neofox), the annotator version, a timestamp and finally a list of the annotations.

In [ ]:
annotations[0].to_dict()

### Transform the annotations into a data frame

In [43]:
annotations_ts = ModelConverter.annotations2tall_skinny_table(neoantigen_annotations=annotations)
annotations_ts.head(10)

,name,neoantigen_identifier,value
0,Best_rank_MHCI_score,ooADEHHbdX//9epH59/tuQ==,6.2429
1,Best_rank_MHCI_score_epitope,ooADEHHbdX//9epH59/tuQ==,GEPSQDILVT
2,Best_rank_MHCI_score_allele,ooADEHHbdX//9epH59/tuQ==,HLA-B*44:03
3,Best_affinity_MHCI_score,ooADEHHbdX//9epH59/tuQ==,3984.4
4,Best_affinity_MHCI_epitope,ooADEHHbdX//9epH59/tuQ==,ILVTDQTRL
5,Best_affinity_MHCI_allele,ooADEHHbdX//9epH59/tuQ==,HLA-C*16:01
6,Best_rank_MHCI_9mer_score,ooADEHHbdX//9epH59/tuQ==,6.2525
7,Best_rank_MHCI_9mer_epitope,ooADEHHbdX//9epH59/tuQ==,ILVTDQTRL
8,Best_rank_MHCI_9mer_allele,ooADEHHbdX//9epH59/tuQ==,HLA-C*16:01
9,Best_affinity_MHCI_9mer_score,ooADEHHbdX//9epH59/tuQ==,3984.4


In [44]:
annotations_sw = ModelConverter.annotations2short_wide_table(annotations, neoantigens)
annotations_sw

,identifier,dnaVariantAlleleFrequency,gene,mutation.mutatedXmer,mutation.position,mutation.wildTypeXmer,patientIdentifier,rnaExpression,rnaVariantAlleleFrequency,ADN_MHCI,...,PHBR-I,PHBR-II,Pathogensimiliarity_MHCI_affinity_9mer,Priority_score,Recognition_Potential_MHCI_affinity_9mer,Selfsimilarity_MHCI_conserved_binder,Tcell_predictor_score_cutoff500nM,mutation_not_found_in_proteome,vaxrank_binding_score,vaxrank_total_score
0,ooADEHHbdX//9epH59/tuQ==,0.0,VCAN,DEVLGEPSQDILVTDQTRLEATISPET,14,DEVLGEPSQDILVIDQTRLEATISPET,Pt27,0.0,0.0,0,...,9.2247,37.098,0,0,NA,0.9867459752059601,NA,1,0,0
1,9kAgxf3dqEc2krSYb30uBQ==,0.0,DCST2,RTNLLAALHRSVRWRAADQGHRSAFLV,14,RTNLLAALHRSVRRRAADQGHRSAFLV,Pt24,0.0,0.0,0,...,3.4198,1.058,0,0,NA,0.9421875787623097,NA,1,0.11069,0
2,s8sKwJW3aw2Bm8vUnElcVg==,0.0,NRAS,MTEYKLVVVGACGVGKSALTIQLIQ,12,MTEYKLVVVGAGGVGKSALTIQLIQ,Pt28,0.0,0.0,0,...,2.4762,35.345,0,0,0,0.9330521460001094,0.5068878716790075,1,1.6547,0
3,iJPwEltDbKAOBzIifaDt0A==,0.0,CEP350,QTDSSSSDMQACSKDKAKISLGSSIDS,14,QTDSSSSDMQACSQDKAKISLGSSIDS,Pt63,0.0,0.0,0,...,4.7855,57.667,0,0,NA,NA,NA,1,0.14174,0
4,wCSx1C7OtJiFgrix/rmMGg==,0.0,CPPED1,DRAIPLVLVSGNHYIGNTPTAETVEEF,14,DRAIPLVLVSGNHDIGNTPTAETVEEF,Pt77,0.0,0.0,1,...,1.4605,7.4182,0,0,0,NA,0.5720939101479084,1,0.89423,0
5,2iu87vd3WXZqJ3ggzlccYg==,0.0,CXorf26,YNKAVYISVQDKEEEKGVNNGGEKRAD,14,YNKAVYISVQDKEGEKGVNNGGEKRAD,Pt117,0.0,0.0,0,...,15.021,29.186,0,0,NA,0.9542175575949939,NA,1,0,0
6,rciXzVxw4LQItjGQZpQAWA==,0.0,IGSF9B,ASTHLTVIGTSPHVPGSVRVQVSMTTA,14,ASTHLTVIGTSPHAPGSVRVQVSMTTA,Pt110,0.0,0.0,0,...,0.93399,24.362,0,0,0,0.9649973487719512,0.09919940654277996,1,3.7569,0
7,Ft7fPmHSZxekWMPk3/gxfg==,0.0,HEATR5A,TRRDEKSHPFTNPQWATRVFAAECVCR,14,TRRDEKSHPFTNPRWATRVFAAECVCR,Pt26,0.0,0.0,0,...,0.74995,1.262,0,0,0,0.9614920042660836,0.3374084622586067,1,2.5678,0
8,zffz2eWQtXHsLzlmXEtU7w==,0.0,CHRDL2,ARPDMFCLFHGKRHFPGESWHPYLEPQ,14,ARPDMFCLFHGKRYFPGESWHPYLEPQ,Pt77,0.0,0.0,0,...,1.6364,31.649,0,0,NA,0.9782021524274523,NA,1,0.92031,0
